In [1]:
import os 
os.chdir ('/home/wenyu/pycharm_projs/Categorical_encoding/')

In [28]:
gp.covar_module.base_kernel.lengthscale

tensor([[1.0548e-32]], dtype=torch.float64, grad_fn=<SoftplusBackward>)

In [27]:
y

tensor([[  -74.8705],
        [  -66.4608],
        [ -853.2452],
        [  -82.3604],
        [  -63.4860],
        [ -338.8379],
        [-2389.6845],
        [ -146.5160],
        [ -201.7074],
        [-1183.8163],
        [  -64.5578],
        [  -83.4013],
        [  -60.7369],
        [  -67.7721],
        [ -585.7819],
        [ -110.7410],
        [  -60.2375],
        [  -74.1374],
        [-2360.8893],
        [-5733.2269],
        [  -74.2981],
        [  -80.6696],
        [  -60.1680],
        [  -65.1957],
        [  -74.1893],
        [-2383.0343],
        [  -78.5776],
        [  -76.3207],
        [  -74.1524],
        [  -86.9156],
        [  -86.4155],
        [  -74.6144],
        [  -85.0600],
        [  -79.9180],
        [  -73.9961],
        [  -64.6920],
        [  -87.3762],
        [  -86.1657],
        [  -65.0430],
        [  -74.0634],
        [  -64.7332],
        [  -80.2885],
        [  -80.1462],
        [  -64.7681],
        [  -83.1616],
        [ 

In [2]:
from initDesignDomain import initBO
from sktools import QuantileEncoder
import syntheticFunctions as test_func
# -------------------START target packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
# -------------------END target packages
import warnings

warnings.filterwarnings("ignore")

# -------------------START Bortorch packages
import torch
import os
import math
import random
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.acquisition.analytic import PosteriorMean
from botorch.acquisition.analytic import ProbabilityOfImprovement
from botorch.acquisition.analytic import UpperConfidenceBound
from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from sklearn import preprocessing
from category_encoders import OneHotEncoder
from category_encoders import TargetEncoder

plt.rcParams['axes.unicode_minus'] = False
print('Libraries imported')

import pickle
import time

Libraries imported


In [3]:
def del_tensor_element(tensor, index):
    t1 = tensor[0:index]
    t2 = tensor[index + 1:]
    return torch.cat((t1, t2), dim=0)

In [4]:
encoder = 'mean'
f = test_func.svm_mse
obj_func = 'svm'
C = [3]
design_seed = 10000
bounds = [
    {'name': 'kernel', 'type': 'categorical', 'domain': tuple(range(3))},
    {'name': 'C', 'type': 'continuous', 'domain': (1, 50)},  # C
    {'name': 'epsilon', 'type': 'continuous', 'domain': (0, 1)}]  # epsilon
max_trials = 20
budget = 200
init_N = 24
df_list = []
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
init_data_path = 'experiment_data/benchmark/init_data/svm/'

In [6]:
start = time.perf_counter()
for trial in range(max_trials):
    print("----------num_trial: ", trial)
    design_obj = initBO(objfn=f, initN=design_seed, bounds=bounds, C=C, saving_path=init_data_path)
    design_data = design_obj.sampling_at_least_once_z(seed=design_seed)
    
    init_obj = initBO(f, init_N, bounds, C, saving_path=init_data_path)
    data, y = init_obj.initialise(seed=trial)
    y_s = scaler.fit_transform(y)

    maxindex_list = []
    cand_y_list = []
    max_y_list = []
    for iteration in range(budget):
        print("iteration: ", iteration)
        TE = TargetEncoder(cols=list(range(len(C)))).fit(data, y_s)
        z = TE.transform(data)
        design_z = torch.from_numpy(TE.transform(design_data).values)

        gp = SingleTaskGP(torch.from_numpy(np.array(z)), y).to(dtype=torch.double)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_model(mll)
        beta = 20 * 0.985 ** iteration 
        UCB = UpperConfidenceBound(model=gp, beta=beta)
        try:
            acq_value = UCB(design_z.unsqueeze(-2))
        except:
            print(f"{trial}_error!!!!!!!!!!!!_{iteration}_iteration")
            break

        max_index = torch.argmax(acq_value, dim=-1).tolist()
        maxindex_list.append(max_index)
        # candidate
        candidate_h_x = design_data[max_index]
#         candidate_y = design_y[max_index].unsqueeze(0)
        candidate_y = torch.tensor(f([candidate_h_x[0]], candidate_h_x[1:])).unsqueeze(0)
        data = np.vstack((data, candidate_h_x))
        y = torch.vstack((y, candidate_y))
        y_s = scaler.transform(y)
#         y_s = scaler.fit_transform(y)
        cand_y_list.append(candidate_y.item())
        max_y_list.append(y.max().item())
        design_data = np.delete(design_data, max_index, axis=0)
#         design_y = del_tensor_element(design_y, max_index)


    final_max = max(max_y_list)
    df_list.append([trial, final_max, cand_y_list, max_y_list])

----------num_trial:  0
self.saving_path: experiment_data/benchmark/init_data/svm/
Creating init data for seed 10000
hinit float64 [[2.]
 [1.]
 [1.]
 ...
 [1.]
 [0.]
 [1.]]
self.saving_path: experiment_data/benchmark/init_data/svm/
init_fname:  False
os.path <module 'posixpath' from '/home/wenyu/.conda/envs/torch/lib/python3.8/posixpath.py'>
Creating init data for seed 0
iteration:  0
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  1
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  2
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  3
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  4
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  5
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  6
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  7
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:

iteration:  101
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  102
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  103
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  104
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  105
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  106
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  107
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  108
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  109
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  110
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  111
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  112
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  113
ard_num_dims=None .!!!!!!!!!!!!!!!!!

iteration:  1
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  2
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  3
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  4
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  5
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  6
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  7
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  8
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  9
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  10
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  11
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  12
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  13
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

iteration:  107
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  108
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  109
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  110
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  111
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  112
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  113
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  114
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  115
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  116
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  117
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  118
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  119
ard_num_dims=None .!!!!!!!!!!!!!!!!!

iteration:  7
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  8
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  9
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  10
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  11
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  12
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  13
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  14
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  15
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  16
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  17
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  18
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  19
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

...

iteration:  92
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  93
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  94
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  95
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  96
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  97
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  98
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  99
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  100
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  101
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  102
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  103
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  104
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!

iteration:  196
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  197
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  198
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  199
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
----------num_trial:  18
self.saving_path: experiment_data/benchmark/init_data/svm/
Using existing init data for seed 10000
self.saving_path: experiment_data/benchmark/init_data/svm/
init_fname:  False
os.path <module 'posixpath' from '/home/wenyu/.conda/envs/torch/lib/python3.8/posixpath.py'>
Creating init data for seed 18
iteration:  0
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  1
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  2
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  3
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  4
ard_num_dims=None .!!!!!!!!!!!!!!!!!

iteration:  97
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  98
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  99
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  100
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  101
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  102
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  103
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  104
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  105
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  106
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  107
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  108
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  109
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!

iteration:  0
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  1
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  2
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  3
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  4
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  5
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  6
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  7
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  8
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  9
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  10
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  11
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  12
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

iteration:  106
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  107
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  108
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  109
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  110
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  111
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  112
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  113
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  114
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  115
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  116
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  117
ard_num_dims=None .!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
iteration:  118
ard_num_dims=None .!!!!!!!!!!!!!!!!!

In [7]:
end = time.perf_counter()
duration = end - start

df = pd.DataFrame(df_list, columns=['num_trial', 'final_max', 'iter_cand_y_list', 'iter_current_max'])
saving_path = f'./experiment_data/benchmark/result/svm/'
file_name = f"{saving_path}{obj_func}_trials_{max_trials}_budget_{budget}_{encoder}_ucbdecay_duration{round(duration)}"
output = open(file_name, 'wb')
pickle.dump(df_list, output)
output.close()
print("-----------duration=", duration)

-----------duration= 2247.8487464260543
